# Equations in OR-Tools

This notebook explores additional ways to model equations in OR Tools.

In [28]:
from ortools.linear_solver import pywraplp
import pandas as pd

In this simple example, we have 3 liquid tanks, each with a different capacity.  Each of the tanks has an amount of starting liquid in it at the start.  We would like the tanks to be as close to 90% full as we can.  We need to choose exactly one of the tanks to add some liquid to, and all of the newly added liquid must go into exactly one of the tanks.  Also each of the tanks has a max level for safety that has been established by the mechanical engineers.

Let's design some data so that when we see the solution we know it did the right thing:

In [20]:
target_fill_ratio = 0.9
tanks_df = pd.DataFrame({
    'capacity': [100, 100, 100],
    'max_level': 95,
    'current_level': [20, 0, 85],
})
tanks_df['target_level'] = target_fill_ratio * tanks_df.capacity
tanks_df['available_volume'] = tanks_df.target_level - tanks_df.current_level
tanks_df['notes'] = [
        'Add 67: comprised of 11 + 56',
        'Add 40',
        'Add 2',
    ]

tanks_df

,capacity,max_level,current_level,target_level,available_volume,notes
0,100,95,20,90.0,70.0,Add 67: comprised of 11 + 56
1,100,95,0,90.0,90.0,Add 40
2,100,95,85,90.0,5.0,Add 2


In [24]:
tank_capacities = [100, 100, 100]
current_fill_levels = [20, 0, 85]
max_fill_levels = [95, 95, 95]
target_fill_ratio = 0.9  # We want the tanks 90% full to the extent we can.
all_tanks = range(len(tank_capacities))

all_demands = [11, 56, 40, 2] # The proposed demands design to fit into particular tanks.
expected_tanks = [0, 0, 1, 2] # The tank numbers we expect these to go to, for checking

In [29]:
model = pywraplp.Solver.CreateSolver("CBC")

In [30]:
# Decision variables
x_vars = {} # x_vars[demand, tank]: For each demand, which tank do we put it in
for demand in all_demands:
    for tank in all_tanks:
        x_vars[demand, tank] = model.BoolVar(f'fill_{demand}_{tank}')


In [33]:
# Constraint: Only one tank is chosen per demand
for demand in all_demands:
    tanks_used = [x_vars[demand, tank] for tank in all_tanks]
    model.Add(sum(tanks_used) == 1)

In [ ]:
# Objective: get tanks close to target level
tank_deviations = []

We want to model `deviation = abs(target_level - fill_level)`, but we are also trying to keep things linear.  `abs()` is not linear, so we need to do a trick to allow us to model this.  The trick is to introduce an extra decision variable for each tank `deviation` and bind it to the absolute value with two inequalities which *are* linear:
* *deviation* >= *expr*
* *deviation* >= *-expr*

Now if we minimize `deviation` that will be equivalent to minimizing `|expr|`.